previously E_J > 1e-3, now change to E_J > 1e-6

In [1]:
import torch
import matplotlib
matplotlib.rcParams.update({'font.size': 11})

import matplotlib.pyplot as plt
from pathlib import Path

from scipy.io import loadmat

datapath = Path('./data/').expanduser()
loaded = torch.load(datapath/'torch_test')
Outputs = loaded['Outputs']
Ns_span = loaded['Ns_span']
rho_span = loaded['rho_span']
Nr = loaded['Nr']
num_try = Outputs.size(dim = 1)

loaded1 = loadmat("./data/allsamples.mat")

C_span = torch.tensor(loaded1["allC"])
G_span = torch.tensor(loaded1["allG"])
S_span = torch.tensor(loaded1["allSstar"])
R_span = torch.tensor(loaded1["allRstar"])

In [2]:
for i in range(len(Ns_span)):
    Ns = Ns_span[i]
    for j in range(len(rho_span)):
        G = G_span[i,j,0:Ns,:]
        C = C_span[i,j,0:Ns,:]
        for k in range(num_try):
            Ss = S_span[i,j,k,:Ns]
            Rs = R_span[i,j,k,:]

            Jstar = torch.zeros(Ns+Nr,Ns+Nr)
            Jstar[0:Ns,Ns:Ns+Nr] = torch.diag(Ss) @ G
            Jstar[Ns:Ns+Nr,0:Ns] = - torch.diag(Rs) @ C.transpose(0,1)
            Jstar[Ns:Ns+Nr,Ns:Ns+Nr] = - torch.diag(C.transpose(0,1) @ Ss)

            E_J = torch.linalg.eigvals(Jstar).real
            Outputs[0,k,i,j] = len(E_J[E_J >= 1.0e-6])/Ns # Fraction of Unstable modes of the real Jacobian
            E_GC = torch.linalg.eigvals(- G @ C.transpose(0,1)).real
            Outputs[1,k,i,j] = len(E_GC[E_GC >= 1.0e-6])/Ns

In [3]:
datapath = Path('./data/').expanduser()
tensorfile = {'Outputs': Outputs, 'Nr': Nr, 'Ns_span': Ns_span, 'rho_span': rho_span}
torch.save(tensorfile, datapath/'torch_testv1.pt')